# train car 2

# 전처리된 훈련, 테스트 데이터를 읽고 학습한다.

In [15]:
import sys
import pandas as pd

from keras.applications import mobilenetv2
from keras.preprocessing import image
# import cv2
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense, Softmax
import random
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import TensorBoard
from keras.models import Input, Model


In [16]:
modelpath='car.h5'
cache=False

x_trainall = np.load('x_train.npy')
y_trainall = np.load('y_train.npy')
# 클래스 데이터 읽기  
dfclass = pd.read_csv('class.csv')
# x_trainall = x_trainall.reshape((-1,224,224,1))  # if gray scale. 

In [17]:
get_ipython().run_line_magic('matplotlib', 'inline')

In [18]:
imgwidth=224
imgheight=224

In [19]:
### shuffle off. already shuffled...
if False:
    idx = np.arange(len(x_trainall))
    np.random.shuffle(idx)
    x_trainall = x_trainall[idx]
    y_trainall = y_trainall[idx]

In [20]:
print(x_trainall.shape, y_trainall.shape)
dfclass.describe()
dfclass.head()

# 클래스는 1~196까지 값을 가짐. 0~196으로 197개클래스로 one-hot을 하기로 함.

(9990, 224, 224, 3) (9990,)


,id,name
0,1,AM General Hummer SUV 2000
1,2,Acura RL Sedan 2012
2,3,Acura TL Sedan 2012
3,4,Acura TL Type-S 2008
4,5,Acura TSX Sedan 2012


In [21]:
x_train = x_trainall[:8000]
y_train = y_trainall[:8000]
x_val = x_trainall[8000:]
y_val = y_trainall[8000:]
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(9000, 224, 224, 3) (9000,) (990, 224, 224, 3) (990,)


In [23]:
print(y_train)
y_train_onehot = np_utils.to_categorical(y_train, 197)
print(y_train_onehot)
y_val_onehot = np_utils.to_categorical(y_val, 197)

[ 24  21 144 ... 186 106  85]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [24]:
tensorboard = TensorBoard(log_dir='log/'+str(time.time()))

if os.path.exists(modelpath):
    cache=True
    print('cache model use... continuous!')
else:
    cache=False
    print('no cache model. create new model!')

In [25]:
if cache==False:
    inputs = Input(shape=(224,224,3))
    net = mobilenetv2.MobileNetV2(input_tensor=inputs, input_shape=(224, 224, 3), alpha=1.0, include_top=False, weights='imagenet', pooling='max')
    net.summary()

In [27]:
# model
if cache==False:
    net2 = Dense(256, activation='relu') (net.layers[-1].output)
    net2 = Dense(197)(net2)
    net2 = Softmax(197)(net2)

    model = Model(inputs=inputs, outputs=net2)
    print('new model...')
else:
    model = load_model(modelpath)
    print('load model...')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
epochs = 200
hist = model.fit( x_train/255., y_train_onehot, batch_size=50, shuffle=True,  epochs=epochs, 
                 verbose=1 , callbacks=[tensorboard], validation_data=[x_val, y_val_onehot] )

Train on 9000 samples, validate on 990 samples
Epoch 1/500
9000/9000 [==============================] - 45s 5ms/step - loss: 0.1689 - acc: 0.9489 - val_loss: 6.5419 - val_acc: 0.2424
Epoch 2/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.2670 - acc: 0.9163 - val_loss: 9.7950 - val_acc: 0.0929
Epoch 3/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.1838 - acc: 0.9430 - val_loss: 5.7134 - val_acc: 0.3525
Epoch 4/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.1308 - acc: 0.9568 - val_loss: 7.2467 - val_acc: 0.2333
Epoch 5/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.1307 - acc: 0.9592 - val_loss: 4.1975 - val_acc: 0.4152
Epoch 6/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0997 - acc: 0.9687 - val_loss: 3.1028 - val_acc: 0.5222
Epoch 7/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.1572 - acc: 0.9507 - val_loss: 3.1913 - val_acc: 0.5182
E

Epoch 120/500
9000/9000 [==============================] - 48s 5ms/step - loss: 0.2050 - acc: 0.9531 - val_loss: 3.2050 - val_acc: 0.6162
Epoch 121/500
9000/9000 [==============================] - 48s 5ms/step - loss: 0.0801 - acc: 0.9790 - val_loss: 2.2948 - val_acc: 0.6919
Epoch 122/500
9000/9000 [==============================] - 48s 5ms/step - loss: 0.0341 - acc: 0.9916 - val_loss: 2.3031 - val_acc: 0.6909
Epoch 123/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0343 - acc: 0.9902 - val_loss: 2.9508 - val_acc: 0.6424
Epoch 124/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0436 - acc: 0.9888 - val_loss: 2.6343 - val_acc: 0.6859
Epoch 125/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0350 - acc: 0.9899 - val_loss: 2.3839 - val_acc: 0.7061
Epoch 126/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0465 - acc: 0.9878 - val_loss: 2.7415 - val_acc: 0.6717
Epoch 127/500
9000/9000 [=========

9000/9000 [==============================] - 49s 5ms/step - loss: 0.0779 - acc: 0.9842 - val_loss: 2.2068 - val_acc: 0.7232
Epoch 239/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.1254 - acc: 0.9750 - val_loss: 3.1713 - val_acc: 0.6485
Epoch 240/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0830 - acc: 0.9819 - val_loss: 2.8449 - val_acc: 0.6798
Epoch 241/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0267 - acc: 0.9930 - val_loss: 2.0307 - val_acc: 0.7687
Epoch 242/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0091 - acc: 0.9972 - val_loss: 1.7716 - val_acc: 0.8010
Epoch 243/500
9000/9000 [==============================] - 46s 5ms/step - loss: 0.0307 - acc: 0.9938 - val_loss: 1.8171 - val_acc: 0.7869
Epoch 244/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0348 - acc: 0.9921 - val_loss: 1.8635 - val_acc: 0.7919
Epoch 245/500
9000/9000 [=======================

9000/9000 [==============================] - 46s 5ms/step - loss: 0.0679 - acc: 0.9879 - val_loss: 2.6050 - val_acc: 0.7465
Epoch 357/500
9000/9000 [==============================] - 46s 5ms/step - loss: 0.0470 - acc: 0.9907 - val_loss: 2.2000 - val_acc: 0.7818
Epoch 358/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.1246 - acc: 0.9794 - val_loss: 2.6870 - val_acc: 0.7071
Epoch 359/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.1028 - acc: 0.9836 - val_loss: 2.3972 - val_acc: 0.7657
Epoch 360/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0350 - acc: 0.9922 - val_loss: 1.9973 - val_acc: 0.8010
Epoch 361/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0244 - acc: 0.9940 - val_loss: 1.8350 - val_acc: 0.8202
Epoch 362/500
9000/9000 [==============================] - 49s 5ms/step - loss: 0.0317 - acc: 0.9936 - val_loss: 2.0395 - val_acc: 0.7828
Epoch 363/500
9000/9000 [=======================

KeyboardInterrupt: 

In [ ]:
hist = model.evaluate( x_val/255., y_val_onehot, batch_size=30, verbose=1 )
print(hist)

In [ ]:
modelpath='car.h5'
model.save(modelpath)

In [ ]:
# submission
x_test = np.load('x_test.npy')
predictions = model.predict( x_test/255. )
pdi = np.argmax(predictions, axis=1)
print(pdi, np.min(pdi), np.max(pdi))


In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission["class"] = pdi
submission.to_csv("submission.csv", index=False)
submission.head()
